In [1]:
import numpy as np
from multiprocessing.dummy import Pool as ThreadPool
import time
from scipy.sparse import csr_matrix
from scipy.sparse.csgraph import shortest_path
import sys
from neuron import NeuronType, Neuron
from util import *
from hyperparams import *
import matplotlib.pyplot as plt
from PIL import Image
import joblib
import gc
import gym
%load_ext autoreload
%autoreload 2

In [2]:
columnIdCount = 0

columnIdCount, V1_levelToColumns, V1_parentEdges, V1_childEdges, V1_siblingEdges = generateColumnTree(V1_numRoots, V1_numLevels, V1_offspring_prob,V1_parent_prob, V1_sibling_prob, V1_mu_offspring, columnIdCount)
V1_num_columns = columnIdCount
V1_columns = range(columnIdCount)
columnIdCount, Thalmus_levelToColumns, Thalmus_parentEdges, Thalmus_childEdges, Thalmus_siblingEdges = generateColumnTree(Thalmus_numRoots, Thalmus_numLevels, Thalmus_offspring_prob,Thalmus_parent_prob, Thalmus_sibling_prob, Thalmus_mu_offspring, columnIdCount)
Thalmus_num_columns = columnIdCount - V1_num_columns
Thalmus_columns = range(V1_num_columns, columnIdCount)
columnIdCount, Motor_levelToColumns, Motor_parentEdges, Motor_childEdges, Motor_siblingEdges = generateColumnTree(Motor_numRoots, Motor_numLevels, Motor_offspring_prob,Motor_parent_prob, Motor_sibling_prob, Motor_mu_offspring,columnIdCount)
Motor_num_columns = columnIdCount - (V1_num_columns + Thalmus_num_columns)
Motor_columns = range(V1_num_columns + Thalmus_num_columns, columnIdCount)

columnIdCount, HC_levelToColumns, HC_parentEdges, HC_childEdges, HC_siblingEdges = generateColumnTree(HC_numRoots, HC_numLevels, HC_offspring_prob,HC_parent_prob, HC_sibling_prob, HC_mu_offspring, columnIdCount)
HC_num_columns = columnIdCount - (V1_num_columns + Thalmus_num_columns + Motor_num_columns)
HC_columns = range(V1_num_columns + Thalmus_num_columns + Motor_num_columns, columnIdCount)


In [3]:
num_layers = 6
prob_by_ntype_by_layer = getNeuronBirthProb(birth_alpha_by_ntype, num_layers)
prob_inhib_by_layer = getInhibNeuronBirthProb(inhibitory_alpha, num_layers)
num_neurons_by_layer = getNumNeuronsByLayer(mu_num_neurons_top, variance_param, mu_num_neurons_other, num_layers)
count_by_ntype_by_layer = getCountByNtypeByLayer(prob_by_ntype_by_layer, num_neurons_by_layer, num_layers)


In [4]:
layerToNeuron = {}
columnToLayer = {}
layerIdToNum = {}
neuronIdToNeuron = {}
trans=[]
neuronId = 0
layerId = 0
columnId = 0
counter = 0
prob_inhib_top_layer = 1.0


#V1
neuronId, layerId, columnId,trans = buildColumnToLayer(V1_columns,prob_inhib_top_layer, prob_inhib_by_layer,count_by_ntype_by_layer, neuronId, layerId, columnId,columnToLayer, layerToNeuron, layerIdToNum,neuronIdToNeuron, trans,num_layers)
neuronId, layerId, columnId,trans = buildColumnToLayer(Thalmus_columns,prob_inhib_top_layer,prob_inhib_by_layer,count_by_ntype_by_layer, neuronId, layerId, columnId,columnToLayer,layerToNeuron, layerIdToNum,neuronIdToNeuron,trans, num_layers)
neuronId, layerId, columnId,trans = buildColumnToLayer(Motor_columns,prob_inhib_top_layer,prob_inhib_by_layer, count_by_ntype_by_layer, neuronId, layerId, columnId,columnToLayer,layerToNeuron, layerIdToNum,neuronIdToNeuron, trans,num_layers)
neuronId, layerId, columnId,trans = buildColumnToLayer(HC_columns,prob_inhib_top_layer,prob_inhib_by_layer, count_by_ntype_by_layer, neuronId, layerId, columnId,columnToLayer,layerToNeuron, layerIdToNum,neuronIdToNeuron, trans,num_layers)
trans = np.array(trans)
assert len(trans) == neuronId
print('Total Neurons: {}, Total Inhib: {}, Total Excite: {}'.format(neuronId,len(np.where(trans < 0)[0]), len(np.where(trans > 0)[0])))

Total Neurons: 37800, Total Inhib: 8349, Total Excite: 29451


In [5]:
memory = 10
p_excite = 0.5
spikes = np.random.choice(a=[True, False], size=(neuronId,memory), p=[p_excite, 1 - p_excite])
v = np.clip(100*np.random.random((neuronId, memory)).astype(np.float16) - 70, -70., 30.)
u = -20*np.random.random((neuronId,memory)).astype(np.float16)
neighbors = np.zeros((neuronId, neuronId), dtype=np.float16)

In [6]:
numConnections = 0
numConnections = connectRegion(V1_columns, V1_parentEdges, V1_siblingEdges, V1_childEdges, columnToLayer,layerIdToNum,sameCol, parentCol,childCol, sibCol, layerToNeuron, neighbors, numConnections)
numConnections = connectRegion(Thalmus_columns, Thalmus_parentEdges, Thalmus_siblingEdges, Thalmus_childEdges, columnToLayer,layerIdToNum,sameCol, parentCol,childCol, sibCol, layerToNeuron, neighbors, numConnections)
numConnections = connectRegion(Motor_columns,Motor_parentEdges, Motor_siblingEdges, Motor_childEdges, columnToLayer,layerIdToNum,sameCol, parentCol,childCol, sibCol, layerToNeuron, neighbors, numConnections)
numConnections = connectRegion(HC_columns, HC_parentEdges, HC_siblingEdges, HC_childEdges, columnToLayer,layerIdToNum,sameCol, parentCol,childCol, sibCol, layerToNeuron, neighbors, numConnections)
print('Total Connections: {}'.format(numConnections))

Added 94222 connections
Added 534222 connections
Added 99418 connections
Added 88116 connections
Total Connections: 815978


In [7]:
input_layer, output_layer, input_layer_inhib = 3, 1, 0

#Generate Vision neurons
vision_neurons = generateVisionNeurons()
print('Total Neurons: {}'.format(np.prod(vision_neurons.shape)))

Total Neurons: 105840


In [8]:
k, stride = 21,9
vision_adj, vision_adj_idx_to_neuronId = connectVisionNeurons(Thalmus_levelToColumns, columnToLayer, layerToNeuron,vision_neurons, k, stride)

In [9]:
# Connect Thalmus to V1
connectRegions(Thalmus_levelToColumns, V1_levelToColumns,outputlevels['TH_V1'], inputlevels['TH_V1'], output_layer, input_layer,columnToLayer, layerToNeuron,neighbors)

#Connect V1 to Thalmus
connectRegions(V1_levelToColumns, Thalmus_levelToColumns, outputlevels['V1_TH'], inputlevels['V1_TH'], output_layer, input_layer_inhib,columnToLayer, layerToNeuron,neighbors)


#Connect HC to V1
connectRegions(HC_levelToColumns, V1_levelToColumns, outputlevels['HC_V1'], inputlevels['HC_V1'], output_layer, input_layer,columnToLayer, layerToNeuron,neighbors)


#Connect V1 to HC-regular
connectRegions(V1_levelToColumns, HC_levelToColumns, outputlevels['V1_HC'], inputlevels['V1_HC'], output_layer, input_layer,columnToLayer, layerToNeuron,neighbors)

#Connect V1 to HC-inhib
connectRegions(V1_levelToColumns, HC_levelToColumns, outputlevels['V1_HC'], inputlevels['V1_HC'], output_layer, input_layer_inhib,columnToLayer, layerToNeuron,neighbors)


#Connect Thalmus to HC
connectRegions(Thalmus_levelToColumns, HC_levelToColumns, outputlevels['TH_HC'], inputlevels['TH_HC'], output_layer, input_layer,columnToLayer, layerToNeuron,neighbors)

#Connect HC to Thalmus
connectRegions(HC_levelToColumns, Thalmus_levelToColumns, outputlevels['HC_TH'], inputlevels['HC_TH'], output_layer, input_layer,columnToLayer, layerToNeuron,neighbors)

#Connect V1 to Motor-reg
connectRegions(V1_levelToColumns, Motor_levelToColumns, outputlevels['V1_MT'], inputlevels['V1_MT'], output_layer, input_layer,columnToLayer, layerToNeuron,neighbors)

#Connect V1 to Motor-inhib
connectRegions(V1_levelToColumns, Motor_levelToColumns, outputlevels['V1_MT'], inputlevels['V1_MT'], output_layer, input_layer_inhib,columnToLayer, layerToNeuron,neighbors)

#Connect Motor to Thalmus
connectRegions(Motor_levelToColumns, Thalmus_levelToColumns, outputlevels['MT_TH'], inputlevels['MT_TH'], output_layer, input_layer,columnToLayer, layerToNeuron,neighbors)

#Connect Thalmus to Motor
connectRegions(Thalmus_levelToColumns, Motor_levelToColumns, outputlevels['TH_MT'], inputlevels['TH_MT'], output_layer, input_layer,columnToLayer, layerToNeuron,neighbors)

#Connect Motor to Thalmus
connectRegions(V1_levelToColumns, Motor_levelToColumns, outputlevels['V1_MT'], inputlevels['V1_MT'], output_layer, input_layer_inhib,columnToLayer, layerToNeuron,neighbors)




Total connections added: 174662
Total connections added: 7147
Total connections added: 12322
Total connections added: 12186
Total connections added: 1148
Total connections added: 158457
Total connections added: 158925
Total connections added: 3674
Total connections added: 315
Total connections added: 47652
Total connections added: 47667
Total connections added: 308


In [10]:
ss = 0.0
for i in range(100):
    idx = np.random.choice(range(25000),1)
    ss+=len(np.nonzero(neighbors[idx])[0])
print(ss/100.)

38.73


In [11]:
imitations = joblib.load('imitations_half.lib')
bins = list(np.arange(0,256,17))
window_size = 3
#Initialize gradient descent data strucuters
dv_dw, dv_dv, idx_to_nid,nid_to_idx = getGradientDataStruc(Motor_columns, columnToLayer,layerToNeuron, window_size)
dv_dw*=np.nan
dv_dv*=np.nan
#Create adjacency dictionary for Motor only
adj_motor = {}
for col in Motor_columns:
    for l in range(num_layers):
        layer = columnToLayer[col][l] #send actions via layer 6
        for neuron_id in layerToNeuron[layer]:
            adj_motor[neuron_id] = np.nonzero(neighbors[neuron_id])[0]


vision_spikes = np.zeros(vision_adj.shape[1], dtype=np.float16)
globalTime = 0

In [ ]:
# Train
for episode in imitations:
    observations, actions = episode[0], episode[1]
    for j in range(len(observations)):
        obs,act = observations[j], actions[j][0]
        idx = globalTime % memory
        #Get vision spikes
        start = time.time()
        vision_spikes = activateVisionNeurons(obs, bins, vision_neurons, vision_spikes)
        #print(np.sum(vision_spikes), np.sum(vision_adj))
        #print('Vision spikes: {}'.format(time.time() - start))
        #Vision Current 
        start = time.time()
        I_vision = np.dot(vision_adj, vision_spikes)
        #print('I_vision: {}'.format(np.sum(I_vision)))
        #print('I_vision: {}'.format(time.time() - start))
        #General cortex
        start = time.time()
        #I = np.dot(neighbors, spikes[:,idx-1])
        I = np.zeros((neighbors.shape[0]),dtype=np.float16)
        #non_zero_spikes = set(np.nonzero(spikes[:,idx-1])[0].tolist())
        non_zero_spikes = np.nonzero(spikes[:,idx-1])[0]
        for spike_col in non_zero_spikes:
            I+= neighbors[:,spike_col]*trans[spike_col]
            #print(np.sum(I))
        
        
        
#         for i in range(neighbors.shape[0]):
#             I_i=0.0
#             neigh = np.nonzero(neighbors[i])[0]
#             for n in neigh:
#                 if n in non_zero_spikes:
#                     I_i+= neighbors[i,n]*spikes[n,idx-1]
#             I[i] = I_i
        #print('I: {}'.format(np.sum(I)))
        #print('I: {}'.format(time.time() - start))
        #Dynamics
        start = time.time()
        dv_dt = 0.04*v[:,idx-1]**2 + 5*v[:,idx-1] + 140 - u[:,idx-1] + I    
        #print('dv_dt: {}'.format(time.time() - start))
        # Add I_vision
        start = time.time()
        for i in range(vision_adj.shape[0]):
            dv_dt[vision_adj_idx_to_neuronId[i]]+=I_vision[i]
        #print('dv_dt[vision_adj_idx_to_neuronId[i]]: {}'.format(time.time() - start))
        
        #Dynamics
        start = time.time()
        du_dt = dynamics[0]["a"]*(dynamics[0]["b"]*v[:,idx-1]- u[:,idx-1])
        #print('du_dt: {}'.format(time.time() - start))
        
        start = time.time()
        v[:,idx] = v[:,idx-1] + lr_params["alpha"]*dv_dt
        #print('Update v: {}'.format(time.time() - start))
        
        start = time.time()
        u[:,idx] = u[:,idx-1] +  lr_params["alpha"]*du_dt
        #print('Update u: {}'.format(time.time() - start))
        
        start = time.time()
        np.clip(v[:,idx], -70.0, 30.0)
        #print('clip v: {}'.format(time.time() - start))

        #Get Spikes
        start = time.time()
        temp_v = v[:,idx]
        spikes[:,idx] = 0
        spikes[np.where(temp_v >= 30),idx] = 1
        print('I_vision: {}, I: {}, Spikes: {}'.format(np.sum(I_vision), np.sum(I), np.sum(spikes[:,idx])))
        #print('generate spikes: {}'.format(time.time() - start))
        #Reset v and u as required
        start = time.time()
        u[temp_v >= 30,idx] +=  dynamics[0]["d"]
        #print('reset u: {}'.format(time.time() - start))
        
        start = time.time()
        v[temp_v>= 30,idx] = dynamics[0]["c"]
        #print('reset v: {}'.format(time.time() - start))
        
        # Get Action
        start = time.time()
        action = getOutput(Motor_levelToColumns,columnToLayer, layerToNeuron,spikes[:,idx], trans,num_classes =3)
        #print('Get output: {}'.format(time.time() - start))
        #Perform STDP
        if (globalTime+1) % 5 == 0:
            start = time.time()
            neighbors=stdp(neighbors, spikes, adj_motor, idx, memory)
            #print('Do STDP: {}'.format(time.time() - start))
        
        
        # Perform gradient descent
        if (globalTime+1) % 10 == 0:
            start = time.time()
            neighbors = gradientDescent(act, Motor_levelToColumns,columnToLayer, layerToNeuron,adj_motor,
                        idx_to_nid, nid_to_idx, spikes, trans, dv_dv, dv_dw, idx, window_size,
                         neighbors, v)
           # print('Do GD: {}'.format(time.time() - start))
        
        globalTime+=1
    

I_vision: 939.5, I: -4812.0, Spikes: 530


In [ ]:
#Generate training data for imitation learning
# obs, act = np.zeros((10000, 84, 84), dtype=np.uint8), np.zeros((10000,1), dtype=np.uint8)
# env = gym.make("Pong-v4")
# #print(env._action_set)
# history_size = 49
# clip_size = 5
# observation = env.reset()
# imitations = []
# counter = 0
# num_frames = 1000000000
# for i in range(1000000000):
#     env.render()
#     action = env.action_space.sample() # your agent here (this takes random actions)
#     observation, reward, done, info = env.step(action)
#     obs[counter] = resizeFrame(observation)
#     act[counter] = action
#     if reward > 0.0:
#         if counter - history_size >= 0:
#             batch = (np.copy(obs[counter-history_size:counter-clip_size]), np.copy(act[counter-history_size:counter-clip_size]))
#         else:
#             batch = (np.copy(obs[:counter+1]), np.copy(act[:counter+1]))
#         imitations.append(batch)
#         print('Idx: {}'.format(counter))
#     counter+=1
#     if counter > 9999:
#         counter = 0
        
#     if done:
#         observation = env.reset()
# env.close()